## Data Science Capstone
#### Mapping and Comparison of two cities to assist in relocation within the U.S

In [3]:
import pandas as pd
import numpy as np
import re 
import itertools
from bs4 import BeautifulSoup
from lxml import html
import requests

from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

#### Function to Scrape City-Data.com

In [1]:
def citypage_scraper(url):

	page = requests.get(url, verify=False).text
	soupobj=BeautifulSoup(page,'lxml')
	
	neighborhoods=[]
	pop_dens=[]
	med_income=[]
	links=[]
	zipcodes=[]
	neighborhood_divs=soupobj.findAll("div",{"class": "neighborhood"})

	for div in neighborhood_divs:
		span=div.find("span", {"class": "street-name"})
		neighborhoods.append(span.text)
		pop_dens.append(div.findAll(text=True)[13].strip())
		med_income.append(div.findAll(text=True)[20].strip()) 
		link=div.findAll('a', href=re.compile('^/neighborhood/'))
		#print(link) find_all('a')[2]["href"]
		links.append(div.find_all('a')[1]["href"])
		
	for link in links:
		fulllink='http://www.city-data.com/'+link
		det_page=BeautifulSoup(requests.get(fulllink).text,'lxml')
		header=det_page.select('h1.city')[0].text.strip()
		zipcodes.append(re.findall(r'\d{5}', header))

	for i, s in enumerate(pop_dens):
		pop_dens[i] = s.replace(',','')
        
	for i, s in enumerate(med_income):
		med_income[i] = s.replace(',', '').replace('$', '')

	return zipcodes,neighborhoods,pop_dens,med_income

In [4]:
#For example, choosing two cities with similar populations
url1='http://www.city-data.com/nbmaps/neigh-Richmond-Virginia.html'
url2='http://www.city-data.com/nbmaps/neigh-Stockton-California.html'
zipcodes1,neighborhoods1,pop_dens1,med_income1=citypage_scraper(url1)
zipcodes2,neighborhoods2,pop_dens2,med_income2=citypage_scraper(url2)

#### Transform City Data into Pandas Dataframe

In [5]:
tuples1=list(zip(zipcodes1, neighborhoods1,pop_dens1,med_income1))
tuples2=list(zip(zipcodes2, neighborhoods2,pop_dens2,med_income2))
df1=pd.DataFrame(tuples1,columns=['ZipCodes','Neighborhood','PopulationDensity','MedianIncome'])
df2=pd.DataFrame(tuples2,columns=['ZipCodes','Neighborhood','PopulationDensity','MedianIncome'])

def melt_series(s):
    lengths = s.str.len().values
    flat = [i for i in itertools.chain.from_iterable(s.values.tolist())]
    idx = np.repeat(s.index.values, lengths)
    return pd.Series(flat, idx, name=s.name)


dfcity1=melt_series(df1.ZipCodes).to_frame().join(df1.drop('ZipCodes', 1)).reindex_axis(df1.columns, 1)
dfcity2=melt_series(df2.ZipCodes).to_frame().join(df2.drop('ZipCodes', 1)).reindex_axis(df2.columns, 1)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:13: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:14: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.


#### Add Latitude and Longitude to City Data Frames

In [6]:
geolocator = Nominatim(user_agent="LocExplorer")

def locfinder(df):
    lats=[]
    longs=[]
    for zipcode in df.ZipCodes:
    #print(zipcode)
        loc=geolocator.geocode(zipcode)
        lats.append(loc.latitude)
        longs.append(loc.longitude)
    return lats,longs

lat1,long1=locfinder(dfcity1)
lat2,long2=locfinder(dfcity2)

In [7]:
#Add Lat Long to DFs
dfcity1['Latitude'] = lat1
dfcity1['Longitude']=long1
dfcity2['Latitude'] = lat2
dfcity2['Longitude']=long2

#Change object type to Int
dfcity1['PopulationDensity']=dfcity1['PopulationDensity'].astype(str).astype(int)
dfcity1['MedianIncome']=dfcity1['MedianIncome'].astype(str).astype(int)

dfcity2['PopulationDensity']=dfcity2['PopulationDensity'].astype(str).astype(int)
dfcity2['MedianIncome']=dfcity2['MedianIncome'].astype(str).astype(int)


#### Map the Two Cities

In [8]:
address='Richmond, VA'
geolocator = Nominatim(user_agent="City_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_city1 = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(dfcity1['Latitude'], dfcity1['Longitude'], dfcity1['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_city1)  
    
map_city1

In [9]:
address2='Stockton, CA'
geolocator = Nominatim(user_agent="City_explorer")
location2 = geolocator.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
map_city2 = folium.Map(location=[latitude2, longitude2], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(dfcity2['Latitude'], dfcity2['Longitude'], dfcity2['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_city2)  
    
map_city2

#### A little quality control, change Population Density and Median Numbers into Categorical Variables

In [10]:
dfcity1.loc[dfcity1.PopulationDensity <= 4000, 'PopDens'] = 'Low' 
dfcity1.loc[(dfcity1.PopulationDensity > 4000) & (dfcity1.PopulationDensity <= 8000), 'PopDens'] = 'Medium' 
dfcity1.loc[dfcity1.PopulationDensity > 10000 , 'PopDens'] = 'High' 

dfcity1.loc[dfcity1.MedianIncome <= 20000, 'MedInc'] = 'Low' 
dfcity1.loc[(dfcity1.MedianIncome > 20000) & (dfcity1.MedianIncome <= 60000), 'MedInc'] = 'Medium' 
dfcity1.loc[dfcity1.MedianIncome > 60000 , 'MedInc'] = 'High' 

dfcity2.loc[dfcity2.PopulationDensity <= 4000, 'PopDens'] = 'Low' 
dfcity2.loc[(dfcity2.PopulationDensity > 4000) & (dfcity2.PopulationDensity <= 8000), 'PopDens'] = 'Medium' 
dfcity2.loc[dfcity2.PopulationDensity > 10000 , 'PopDens'] = 'High' 

dfcity2.loc[dfcity2.MedianIncome <= 20000, 'MedInc'] = 'Low' 
dfcity2.loc[(dfcity2.MedianIncome > 20000) & (dfcity2.MedianIncome <= 60000), 'MedInc'] = 'Medium' 
dfcity2.loc[dfcity2.MedianIncome > 60000 , 'MedInc'] = 'High' 


dfcity1=dfcity1.drop(['PopulationDensity', 'MedianIncome'], axis=1)
dfcity2=dfcity2.drop(['PopulationDensity', 'MedianIncome'], axis=1)



#### Explore venues using FourSquare API

In [11]:
CLIENT_ID = 'ZZZZZZZZ' # your Foursquare ID
CLIENT_SECRET = 'XXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
limit=100
radius=500

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
city1_venues = getNearbyVenues(names=dfcity1['Neighborhood'],
                                   latitudes=dfcity1['Latitude'],
                                   longitudes=dfcity1['Longitude']
                                  )

Barton Heights
Barton Heights Cemeteries
Battery Park
Belle Isle
Bellevue
Blackwell
Boulevard
Boulevard
Broad Rock
Broad Rock
Broad Street Commercial
Broad Street Commercial
Brookland Park
Capital Square
Carver
Chestnut Hill
Chimborazo Park
Church Hill Central
Church Hill North
Commonwealth Club
Court End
Court End
Downtown
Downtown
East
East
East
East Franklin Street
East Franklin Street
Fan
Fan Extension
Far West
Far West
Far West
Forest Hill Park
Ginter Park
Ginter Park
Grace Street Commercial
Hermitage Road
Hollywood Cemetery
Huguenot
Huguenot
Jackson Ward
Jackson Ward
Laburnum Court
Manchester
Midlothian
Midlothian
Monroe Park
Monroe Ward
Monroe Ward
Monument Avenue
Monument Avenue
Museum District
Near West
Near West
Near West
North
North
North
Old South
Old South
Oregon Hill
Shockoe Bottom
Shockoe Slip
Springhill
St. John's Church
Tobacco Row
Tobacco Row
Two Hundred Block West Franklin
Union Hill
West Broad Street
West Franklin Street
West Grace Street
Woodland Heights


In [14]:
city2_venues = getNearbyVenues(names=dfcity2['Neighborhood'],
                                   latitudes=dfcity2['Latitude'],
                                   longitudes=dfcity2['Longitude']
                                  )

Airport
Airport
Airport
Akers
Akers
Anderson
Anderson
August
Bear Creek District
Bear Creek District
Bear Creek District
Brookside
Civic District
Civic District
Civic District
Civic District
Civic District
Colonial Heights
Colonial Heights
Colonial Heights
Country Club
Creekside/Wagner
Creekside/Wagner
Downtown
Downtown
Downtown
East Stockton
East Stockton
East Stockton
East Stockton
East Stockton
East Stockton
El Pinal
El Pinal
Elkhorn
Elkhorn
Elkhorn
Enclave at Spanos Park East
Enclave at Spanos Park East
Fair Oaks
Fair Oaks
Fox Creek
Fox Creek
Fremont Park
Garden Acres
Garden Acres
Gleason Park
Gleason Park
Gleason Park
Holiday Park
Holiday Park
Kennedy
Kennedy
Kentfield
Kentfield
La Morada
Lakeview District
Lakeview District
Lincoln Village West
Louis Park
Louis Park
Magnolia
Mariposa
Mariposa
McKinley Park
McKinley Park
Midtown
Midtown West
Midtown West
Mormon
North Stockton
North Stockton
North Stockton
North Stockton
North Stockton
North Stockton
North Stockton
North Stockton
No

In [15]:
city1_venues.groupby('Neighborhood').count()
city2_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Airport,4,4,4,4,4,4
Akers,24,24,24,24,24,24
Anderson,24,24,24,24,24,24
August,2,2,2,2,2,2
Bear Creek District,13,13,13,13,13,13
Brookside,1,1,1,1,1,1
Civic District,34,34,34,34,34,34
Colonial Heights,17,17,17,17,17,17
Country Club,2,2,2,2,2,2


#### One Hot Encoding for Neighborhoods

In [16]:
#Function to one hot encode
def onehot(df):
    df_onehot=pd.get_dummies(df[['Venue Category']], prefix="", prefix_sep="")
    
    df_onehot['Neighborhood']=df['Neighborhood']
   
    fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
    df_onehot = df_onehot[fixed_columns]

    return df_onehot





In [17]:
#Run cities
city1onehot=onehot(city1_venues)
city2onehot=onehot(city2_venues)

city1_grouped = city1onehot.groupby('Neighborhood').mean().reset_index()
city2_grouped = city2onehot.groupby('Neighborhood').mean().reset_index()


city2_grouped

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bank,Big Box Store,Bowling Alley,Bubble Tea Shop,Burger Joint,...,Rental Car Location,Sandwich Place,Seafood Restaurant,Smoke Shop,Supermarket,Taco Place,Tailor Shop,Theater,Turkish Restaurant,Wine Shop
0,Airport,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000
1,Akers,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.041667,0.041667,0.000000,...,0.000000,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.000000,0.041667,0.041667
2,Anderson,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.041667,0.041667,0.000000,...,0.000000,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.000000,0.041667,0.041667
3,August,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000
4,Bear Creek District,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000
5,Brookside,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Civic District,0.000000,0.000000,0.000000,0.029412,0.058824,0.000000,0.000000,0.000000,0.029412,...,0.058824,0.029412,0.000000,0.029412,0.000000,0.029412,0.029412,0.029412,0.000000,0.000000
7,Colonial Heights,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,...,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
8,Country Club,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Creekside/Wagner,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city1_venues_sorted = pd.DataFrame(columns=columns)
city1_venues_sorted['Neighborhood'] = city1_grouped['Neighborhood']

for ind in np.arange(city1_grouped.shape[0]):
    city1_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city1_grouped.iloc[ind, :], num_top_venues)

In [ ]:
city1_venues_sorted.head()

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city2_venues_sorted = pd.DataFrame(columns=columns)
city2_venues_sorted['Neighborhood'] = city2_grouped['Neighborhood']

for ind in np.arange(city2_grouped.shape[0]):
    city2_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city2_grouped.iloc[ind, :], num_top_venues)

In [ ]:
city2_venues_sorted.head()

#### Clustering Neighborhoods

In [21]:
kclusters = 12

city1_grouped_clustering = city1_grouped.drop('Neighborhood', 1)

city1_grouped_clustering
# # run k-means clustering
kmeans1 = KMeans(n_clusters=kclusters, random_state=0).fit(city1_grouped_clustering)
#kmeans.labels_[0:5] 
kmeans1.labels_

array([ 0,  4, 11,  9, 11,  7,  3,  0,  2,  2,  2,  0, 10,  7, 10,  7,  0,
        0,  8,  1,  4,  3,  4,  0,  1,  7,  4, 11,  5,  0,  7,  9,  6,  9,
        9,  5,  0,  2,  3,  1,  2, 10,  0,  2,  0,  0,  0,  1], dtype=int32)

In [22]:
kclusters = 12

city2_grouped_clustering = city2_grouped.drop('Neighborhood', 1)

city2_grouped_clustering
# # run k-means clustering
kmeans2 = KMeans(n_clusters=kclusters, random_state=0).fit(city2_grouped_clustering)
#kmeans.labels_[0:5] 
kmeans2.labels_

array([ 5,  6,  6,  2,  1,  3,  8,  6,  0,  7,  8, 10,  9,  1,  7,  5, 10,
        8,  5, 11,  6,  2,  6, 10,  6,  3, 11,  8,  5,  4,  8,  8,  2,  6,
       10,  6,  0,  5,  6,  1,  6,  8, 11,  9,  9,  4,  3,  7,  6,  1,  6,
        8, 10,  8, 10,  6,  6,  4,  2], dtype=int32)

In [23]:
# add clustering labels
city1_venues_sorted.insert(0, 'Cluster Labels', kmeans1.labels_)

# # neighborhoods_venues_sorted
city1_merged = dfcity1

# # # # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
city1_merged = city1_merged.join(city1_venues_sorted.set_index('Neighborhood'), how='right',on='Neighborhood')

city1_merged.head() # check the last columns!

,ZipCodes,Neighborhood,Latitude,Longitude,PopDens,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,23220,Belle Isle,37.553254,-77.445504,Low,Medium,0,Coffee Shop,Donut Shop,Mexican Restaurant,Bookstore,Nightclub,Thrift / Vintage Store,Fish Market,Fast Food Restaurant,Farmers Market,Dry Cleaner
4,23227,Bellevue,37.603410,-77.446582,Low,Medium,4,Casino,Fried Chicken Joint,Discount Store,Southern / Soul Food Restaurant,Chinese Restaurant,Thrift / Vintage Store,Art Gallery,Food Truck,Food,Fish Market
5,23224,Blackwell,37.519435,-77.419623,Low,Medium,11,Park,Harbor / Marina,Volleyball Court,Caribbean Restaurant,Grocery Store,Seafood Restaurant,Donut Shop,Deli / Bodega,Diner,Discount Store
6,23220,Boulevard,37.553254,-77.445504,NaN,Medium,9,Coffee Shop,Nightclub,Bookstore,Donut Shop,Mexican Restaurant,Garden,Thrift / Vintage Store,Speakeasy,Auto Dealership,Food Truck
6,23221,Boulevard,37.562376,-77.483962,NaN,Medium,9,Coffee Shop,Nightclub,Bookstore,Donut Shop,Mexican Restaurant,Garden,Thrift / Vintage Store,Speakeasy,Auto Dealership,Food Truck


In [24]:
city2_venues_sorted.insert(0, 'Cluster Labels', kmeans2.labels_)

# # neighborhoods_venues_sorted
city2_merged = dfcity2

# # # # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
city2_merged = city2_merged.join(city2_venues_sorted.set_index('Neighborhood'), how='right',on='Neighborhood')

city2_merged.head() # check the last columns!

,ZipCodes,Neighborhood,Latitude,Longitude,PopDens,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,95205,Airport,37.974233,-121.267864,Low,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
0,95206,Airport,37.930699,-121.275570,Low,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
0,95215,Airport,37.958887,-121.237743,Low,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
1,95207,Akers,37.988492,-121.332081,Medium,High,6,Japanese Restaurant,Pizza Place,Wine Shop,Mexican Restaurant,Fast Food Restaurant,Financial or Legal Service,Convention Center,French Restaurant,Turkish Restaurant,Gym / Fitness Center
1,95210,Akers,48.968175,2.284591,Medium,High,6,Japanese Restaurant,Pizza Place,Wine Shop,Mexican Restaurant,Fast Food Restaurant,Financial or Legal Service,Convention Center,French Restaurant,Turkish Restaurant,Gym / Fitness Center


In [25]:
map_clusters1 = folium.Map(location=[37.540726,-77.436050], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city1_merged['Latitude'], city1_merged['Longitude'], city1_merged['Neighborhood'], city1_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters1)
       
map_clusters1

In [26]:
map_clusters2 = folium.Map(location=[37.957703,-121.290779], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city2_merged['Latitude'], city2_merged['Longitude'], city2_merged['Neighborhood'], city2_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters2)
       
map_clusters2

In [27]:
city1_merged.loc[city1_merged['Cluster Labels'] == 0, city1_merged.columns[[1] + list(range(5, city1_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Belle Isle,Medium,0,Coffee Shop,Donut Shop,Mexican Restaurant,Bookstore,Nightclub,Thrift / Vintage Store,Fish Market,Fast Food Restaurant,Farmers Market,Dry Cleaner
11,Carver,Medium,0,Coffee Shop,Donut Shop,Mexican Restaurant,Bookstore,Nightclub,Thrift / Vintage Store,Fish Market,Fast Food Restaurant,Farmers Market,Dry Cleaner
16,Commonwealth Club,Medium,0,Coffee Shop,Donut Shop,Mexican Restaurant,Bookstore,Nightclub,Thrift / Vintage Store,Fish Market,Fast Food Restaurant,Farmers Market,Dry Cleaner
21,Fan,Medium,0,Coffee Shop,Donut Shop,Mexican Restaurant,Bookstore,Nightclub,Thrift / Vintage Store,Fish Market,Fast Food Restaurant,Farmers Market,Dry Cleaner
22,Fan Extension,Medium,0,Coffee Shop,Donut Shop,Mexican Restaurant,Bookstore,Nightclub,Thrift / Vintage Store,Fish Market,Fast Food Restaurant,Farmers Market,Dry Cleaner
28,Hollywood Cemetery,Medium,0,Coffee Shop,Donut Shop,Mexican Restaurant,Bookstore,Nightclub,Thrift / Vintage Store,Fish Market,Fast Food Restaurant,Farmers Market,Dry Cleaner
34,Monroe Park,Low,0,Coffee Shop,Donut Shop,Mexican Restaurant,Bookstore,Nightclub,Thrift / Vintage Store,Fish Market,Fast Food Restaurant,Farmers Market,Dry Cleaner
41,Oregon Hill,Medium,0,Coffee Shop,Donut Shop,Mexican Restaurant,Bookstore,Nightclub,Thrift / Vintage Store,Fish Market,Fast Food Restaurant,Farmers Market,Dry Cleaner
47,Two Hundred Block West Franklin,Medium,0,Coffee Shop,Donut Shop,Mexican Restaurant,Bookstore,Nightclub,Thrift / Vintage Store,Fish Market,Fast Food Restaurant,Farmers Market,Dry Cleaner
49,West Broad Street,Medium,0,Coffee Shop,Donut Shop,Mexican Restaurant,Bookstore,Nightclub,Thrift / Vintage Store,Fish Market,Fast Food Restaurant,Farmers Market,Dry Cleaner


In [28]:
city2_merged.loc[city2_merged['Cluster Labels'] == 0, city2_merged.columns[[1] + list(range(5, city2_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Country Club,High,0,Business Service,College Baseball Diamond,Wine Shop,Coffee Shop,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie
36,Oak Park,Medium,0,Business Service,College Baseball Diamond,Wine Shop,Coffee Shop,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie


In [29]:
city1_merged.loc[city1_merged['Cluster Labels'] == 1, city1_merged.columns[[1] + list(range(5, city1_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Forest Hill Park,Medium,1,Park,Dog Run,Art Gallery,Business Service,Farmers Market,Music Venue,College Gym,Deli / Bodega,Diner,Discount Store
29,Huguenot,High,1,Park,Dog Run,Art Gallery,Business Service,Farmers Market,Music Venue,College Gym,Deli / Bodega,Diner,Discount Store
29,Huguenot,High,1,Park,Dog Run,Art Gallery,Business Service,Farmers Market,Music Venue,College Gym,Deli / Bodega,Diner,Discount Store
44,Springhill,Medium,1,Park,Dog Run,Art Gallery,Business Service,Farmers Market,Music Venue,College Gym,Deli / Bodega,Diner,Discount Store
52,Woodland Heights,Medium,1,Park,Dog Run,Art Gallery,Business Service,Farmers Market,Music Venue,College Gym,Deli / Bodega,Diner,Discount Store


In [30]:
city2_merged.loc[city2_merged['Cluster Labels'] == 1, city2_merged.columns[[1] + list(range(5, city2_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Bear Creek District,High,1,Japanese Restaurant,Food,Convention Center,Art Gallery,French Restaurant,Clothing Store,Supermarket,Turkish Restaurant,Bowling Alley,Pool Hall
4,Bear Creek District,High,1,Japanese Restaurant,Food,Convention Center,Art Gallery,French Restaurant,Clothing Store,Supermarket,Turkish Restaurant,Bowling Alley,Pool Hall
4,Bear Creek District,High,1,Japanese Restaurant,Food,Convention Center,Art Gallery,French Restaurant,Clothing Store,Supermarket,Turkish Restaurant,Bowling Alley,Pool Hall
13,Elkhorn,High,1,Japanese Restaurant,French Restaurant,Clothing Store,Food,Convention Center,Pizza Place,Creperie,Bowling Alley,Supermarket,Turkish Restaurant
13,Elkhorn,High,1,Japanese Restaurant,French Restaurant,Clothing Store,Food,Convention Center,Pizza Place,Creperie,Bowling Alley,Supermarket,Turkish Restaurant
13,Elkhorn,High,1,Japanese Restaurant,French Restaurant,Clothing Store,Food,Convention Center,Pizza Place,Creperie,Bowling Alley,Supermarket,Turkish Restaurant
40,Ponce de Leon,Medium,1,Japanese Restaurant,French Restaurant,Clothing Store,Food,Convention Center,Pizza Place,Creperie,Bowling Alley,Supermarket,Turkish Restaurant
40,Ponce de Leon,Medium,1,Japanese Restaurant,French Restaurant,Clothing Store,Food,Convention Center,Pizza Place,Creperie,Bowling Alley,Supermarket,Turkish Restaurant
50,Tam O'Shanter,Medium,1,Japanese Restaurant,French Restaurant,Turkish Restaurant,Supermarket,Bowling Alley,Creperie,Pizza Place,Convention Center,Clothing Store,College Baseball Diamond


In [31]:
city1_merged.loc[city1_merged['Cluster Labels'] == 2, city1_merged.columns[[1] + list(range(5, city1_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Chimborazo Park,High,2,Grocery Store,Deli / Bodega,German Restaurant,BBQ Joint,Gift Shop,Southern / Soul Food Restaurant,Bakery,Pizza Place,Playground,Donut Shop
14,Church Hill Central,Medium,2,Grocery Store,Deli / Bodega,German Restaurant,BBQ Joint,Gift Shop,Southern / Soul Food Restaurant,Bakery,Pizza Place,Playground,Donut Shop
15,Church Hill North,Medium,2,Grocery Store,Deli / Bodega,German Restaurant,BBQ Joint,Gift Shop,Southern / Soul Food Restaurant,Bakery,Pizza Place,Playground,Donut Shop
42,Shockoe Bottom,Medium,2,Grocery Store,Deli / Bodega,German Restaurant,BBQ Joint,Gift Shop,Southern / Soul Food Restaurant,Bakery,Pizza Place,Playground,Donut Shop
45,St. John's Church,Medium,2,Grocery Store,Deli / Bodega,German Restaurant,BBQ Joint,Gift Shop,Southern / Soul Food Restaurant,Bakery,Pizza Place,Playground,Donut Shop
48,Union Hill,Medium,2,Grocery Store,Deli / Bodega,German Restaurant,BBQ Joint,Gift Shop,Southern / Soul Food Restaurant,Bakery,Pizza Place,Playground,Donut Shop


In [32]:
city2_merged.loc[city2_merged['Cluster Labels'] == 2, city2_merged.columns[[1] + list(range(5, city2_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,August,Medium,2,Gas Station,Taco Place,Clothing Store,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
21,Kennedy,Medium,2,Gas Station,Taco Place,Clothing Store,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
21,Kennedy,Medium,2,Gas Station,Taco Place,Clothing Store,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
32,Mormon,Medium,2,Gas Station,Taco Place,Clothing Store,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
60,Wilson,Medium,2,Gas Station,Taco Place,Clothing Store,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop


In [33]:
city1_merged.loc[city1_merged['Cluster Labels'] == 3, city1_merged.columns[[1] + list(range(5, city1_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Capital Square,Medium,3,Coffee Shop,College Cafeteria,Music Venue,College Gym,Café,Garden,Food,Food Truck,Fast Food Restaurant,Farmers Market
26,Grace Street Commercial,Medium,3,Coffee Shop,College Cafeteria,Music Venue,College Gym,Café,Garden,Food,Food Truck,Fast Food Restaurant,Farmers Market
43,Shockoe Slip,Medium,3,Coffee Shop,College Cafeteria,Music Venue,College Gym,Café,Garden,Food,Food Truck,Fast Food Restaurant,Farmers Market


In [34]:
city2_merged.loc[city2_merged['Cluster Labels'] == 3, city2_merged.columns[[1] + list(range(5, city2_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Brookside,High,3,Pool Hall,Wine Shop,Clothing Store,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
25,Lincoln Village West,High,3,Pool Hall,Wine Shop,Clothing Store,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
47,Spanos Park West,High,3,Pool Hall,Wine Shop,Clothing Store,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop


In [35]:
city1_merged.loc[city1_merged['Cluster Labels'] == 4, city1_merged.columns[[1] + list(range(5, city1_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Bellevue,Medium,4,Casino,Fried Chicken Joint,Discount Store,Southern / Soul Food Restaurant,Chinese Restaurant,Thrift / Vintage Store,Art Gallery,Food Truck,Food,Fish Market
25,Ginter Park,Medium,4,Casino,Fried Chicken Joint,Discount Store,Southern / Soul Food Restaurant,Chinese Restaurant,Thrift / Vintage Store,Art Gallery,Food Truck,Food,Fish Market
25,Ginter Park,Medium,4,Casino,Fried Chicken Joint,Discount Store,Southern / Soul Food Restaurant,Chinese Restaurant,Thrift / Vintage Store,Art Gallery,Food Truck,Food,Fish Market
27,Hermitage Road,Medium,4,Casino,Fried Chicken Joint,Discount Store,Southern / Soul Food Restaurant,Chinese Restaurant,Thrift / Vintage Store,Art Gallery,Food Truck,Food,Fish Market
31,Laburnum Court,High,4,Casino,Fried Chicken Joint,Discount Store,Southern / Soul Food Restaurant,Chinese Restaurant,Thrift / Vintage Store,Art Gallery,Food Truck,Food,Fish Market


In [36]:
city2_merged.loc[city2_merged['Cluster Labels'] == 4, city2_merged.columns[[1] + list(range(5, city2_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,McKinley Park,Medium,4,Café,Wine Shop,Coffee Shop,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
29,McKinley Park,Medium,4,Café,Wine Shop,Coffee Shop,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
46,Southwest Stockton,Medium,4,Café,Wine Shop,Coffee Shop,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
46,Southwest Stockton,Medium,4,Café,Wine Shop,Coffee Shop,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
59,Weston Ranch,Medium,4,Café,Wine Shop,Coffee Shop,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
59,Weston Ranch,Medium,4,Café,Wine Shop,Coffee Shop,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop


In [37]:
city1_merged.loc[city1_merged['Cluster Labels'] == 5, city1_merged.columns[[1] + list(range(5, city1_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Midlothian,Medium,5,Park,Dog Run,Art Gallery,Music Venue,Volleyball Court,Farmers Market,Seafood Restaurant,Grocery Store,Harbor / Marina,Caribbean Restaurant
33,Midlothian,Medium,5,Park,Dog Run,Art Gallery,Music Venue,Volleyball Court,Farmers Market,Seafood Restaurant,Grocery Store,Harbor / Marina,Caribbean Restaurant
40,Old South,Medium,5,Park,Dog Run,Art Gallery,Music Venue,Volleyball Court,Farmers Market,Seafood Restaurant,Grocery Store,Harbor / Marina,Caribbean Restaurant
40,Old South,Medium,5,Park,Dog Run,Art Gallery,Music Venue,Volleyball Court,Farmers Market,Seafood Restaurant,Grocery Store,Harbor / Marina,Caribbean Restaurant


In [38]:
city2_merged.loc[city2_merged['Cluster Labels'] == 5, city2_merged.columns[[1] + list(range(5, city2_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Airport,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
0,Airport,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
0,Airport,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
15,Fair Oaks,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
15,Fair Oaks,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
18,Garden Acres,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
18,Garden Acres,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
28,Mariposa,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
28,Mariposa,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop
38,Park District,Medium,5,Gas Station,Grocery Store,Taco Place,Clothing Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop


In [39]:
city1_merged.loc[city1_merged['Cluster Labels'] == 6, city1_merged.columns[[1] + list(range(5, city1_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Museum District,Medium,6,Garden,Speakeasy,Donut Shop,College Cafeteria,College Gym,Deli / Bodega,Diner,Discount Store,Dog Run,Wine Shop


In [40]:
city2_merged.loc[city2_merged['Cluster Labels'] == 6, city2_merged.columns[[1] + list(range(5, city2_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Akers,High,6,Japanese Restaurant,Pizza Place,Wine Shop,Mexican Restaurant,Fast Food Restaurant,Financial or Legal Service,Convention Center,French Restaurant,Turkish Restaurant,Gym / Fitness Center
1,Akers,High,6,Japanese Restaurant,Pizza Place,Wine Shop,Mexican Restaurant,Fast Food Restaurant,Financial or Legal Service,Convention Center,French Restaurant,Turkish Restaurant,Gym / Fitness Center
2,Anderson,Medium,6,Japanese Restaurant,Pizza Place,Wine Shop,Mexican Restaurant,Fast Food Restaurant,Financial or Legal Service,Convention Center,French Restaurant,Turkish Restaurant,Gym / Fitness Center
2,Anderson,Medium,6,Japanese Restaurant,Pizza Place,Wine Shop,Mexican Restaurant,Fast Food Restaurant,Financial or Legal Service,Convention Center,French Restaurant,Turkish Restaurant,Gym / Fitness Center
7,Colonial Heights,High,6,Wine Shop,Pizza Place,Financial or Legal Service,Food,Gym / Fitness Center,Indian Restaurant,Fast Food Restaurant,Mexican Restaurant,Optical Shop,Pet Store
7,Colonial Heights,High,6,Wine Shop,Pizza Place,Financial or Legal Service,Food,Gym / Fitness Center,Indian Restaurant,Fast Food Restaurant,Mexican Restaurant,Optical Shop,Pet Store
7,Colonial Heights,High,6,Wine Shop,Pizza Place,Financial or Legal Service,Food,Gym / Fitness Center,Indian Restaurant,Fast Food Restaurant,Mexican Restaurant,Optical Shop,Pet Store
20,Holiday Park,Medium,6,Japanese Restaurant,Pizza Place,Wine Shop,Mexican Restaurant,Fast Food Restaurant,Financial or Legal Service,Convention Center,French Restaurant,Turkish Restaurant,Gym / Fitness Center
20,Holiday Park,Medium,6,Japanese Restaurant,Pizza Place,Wine Shop,Mexican Restaurant,Fast Food Restaurant,Financial or Legal Service,Convention Center,French Restaurant,Turkish Restaurant,Gym / Fitness Center
22,Kentfield,Medium,6,Japanese Restaurant,Pizza Place,Wine Shop,Mexican Restaurant,Fast Food Restaurant,Financial or Legal Service,Convention Center,French Restaurant,Turkish Restaurant,Gym / Fitness Center


In [41]:
city1_merged.loc[city1_merged['Cluster Labels'] == 7, city1_merged.columns[[1] + list(range(5, city1_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Broad Street Commercial,Medium,7,Coffee Shop,Nightclub,Bookstore,Food Truck,Food,Donut Shop,College Gym,Mexican Restaurant,College Cafeteria,Music Venue
8,Broad Street Commercial,Medium,7,Coffee Shop,Nightclub,Bookstore,Food Truck,Food,Donut Shop,College Gym,Mexican Restaurant,College Cafeteria,Music Venue
18,Downtown,Medium,7,Coffee Shop,Nightclub,Bookstore,Food Truck,Food,Donut Shop,College Gym,Mexican Restaurant,College Cafeteria,Music Venue
18,Downtown,Medium,7,Coffee Shop,Nightclub,Bookstore,Food Truck,Food,Donut Shop,College Gym,Mexican Restaurant,College Cafeteria,Music Venue
20,East Franklin Street,Medium,7,Coffee Shop,Nightclub,Bookstore,Food Truck,Food,Donut Shop,College Gym,Mexican Restaurant,College Cafeteria,Music Venue
20,East Franklin Street,Medium,7,Coffee Shop,Nightclub,Bookstore,Food Truck,Food,Donut Shop,College Gym,Mexican Restaurant,College Cafeteria,Music Venue
30,Jackson Ward,Medium,7,Coffee Shop,Nightclub,Bookstore,Food Truck,Food,Donut Shop,College Gym,Mexican Restaurant,College Cafeteria,Music Venue
30,Jackson Ward,Medium,7,Coffee Shop,Nightclub,Bookstore,Food Truck,Food,Donut Shop,College Gym,Mexican Restaurant,College Cafeteria,Music Venue
35,Monroe Ward,Medium,7,Coffee Shop,Nightclub,Bookstore,Food Truck,Food,Donut Shop,College Gym,Mexican Restaurant,College Cafeteria,Music Venue
35,Monroe Ward,Medium,7,Coffee Shop,Nightclub,Bookstore,Food Truck,Food,Donut Shop,College Gym,Mexican Restaurant,College Cafeteria,Music Venue


In [42]:
city2_merged.loc[city2_merged['Cluster Labels'] == 7, city2_merged.columns[[1] + list(range(5, city2_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Creekside/Wagner,High,7,Art Gallery,Food,Pool Hall,Wine Shop,Coffee Shop,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie
9,Creekside/Wagner,High,7,Art Gallery,Food,Pool Hall,Wine Shop,Coffee Shop,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie
14,Enclave at Spanos Park East,High,7,Art Gallery,Food,Pool Hall,Wine Shop,Coffee Shop,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie
14,Enclave at Spanos Park East,High,7,Art Gallery,Food,Pool Hall,Wine Shop,Coffee Shop,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie
48,Stonewood,High,7,Art Gallery,Food,Wine Shop,Coffee Shop,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store,Creperie,Cosmetics Shop


In [43]:
city1_merged.loc[city1_merged['Cluster Labels'] == 8, city1_merged.columns[[1] + list(range(5, city1_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Far West,High,8,American Restaurant,Pizza Place,Athletics & Sports,Bakery,Bookstore,Breakfast Spot,Chinese Restaurant,Diner,Dry Cleaner,Fish Market
23,Far West,High,8,American Restaurant,Pizza Place,Athletics & Sports,Bakery,Bookstore,Breakfast Spot,Chinese Restaurant,Diner,Dry Cleaner,Fish Market
23,Far West,High,8,American Restaurant,Pizza Place,Athletics & Sports,Bakery,Bookstore,Breakfast Spot,Chinese Restaurant,Diner,Dry Cleaner,Fish Market


In [44]:
city2_merged.loc[city2_merged['Cluster Labels'] == 8, city2_merged.columns[[1] + list(range(5, city2_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Civic District,Medium,8,Mexican Restaurant,Gas Station,Coffee Shop,Harbor / Marina,Bank,Rental Car Location,Food,College Baseball Diamond,Greek Restaurant,Japanese Restaurant
6,Civic District,Medium,8,Mexican Restaurant,Gas Station,Coffee Shop,Harbor / Marina,Bank,Rental Car Location,Food,College Baseball Diamond,Greek Restaurant,Japanese Restaurant
6,Civic District,Medium,8,Mexican Restaurant,Gas Station,Coffee Shop,Harbor / Marina,Bank,Rental Car Location,Food,College Baseball Diamond,Greek Restaurant,Japanese Restaurant
6,Civic District,Medium,8,Mexican Restaurant,Gas Station,Coffee Shop,Harbor / Marina,Bank,Rental Car Location,Food,College Baseball Diamond,Greek Restaurant,Japanese Restaurant
6,Civic District,Medium,8,Mexican Restaurant,Gas Station,Coffee Shop,Harbor / Marina,Bank,Rental Car Location,Food,College Baseball Diamond,Greek Restaurant,Japanese Restaurant
10,Downtown,Medium,8,Mexican Restaurant,Coffee Shop,Harbor / Marina,Rental Car Location,Bank,Gas Station,Filipino Restaurant,Convenience Store,Food,Greek Restaurant
10,Downtown,Medium,8,Mexican Restaurant,Coffee Shop,Harbor / Marina,Rental Car Location,Bank,Gas Station,Filipino Restaurant,Convenience Store,Food,Greek Restaurant
10,Downtown,Medium,8,Mexican Restaurant,Coffee Shop,Harbor / Marina,Rental Car Location,Bank,Gas Station,Filipino Restaurant,Convenience Store,Food,Greek Restaurant
17,Fremont Park,Low,8,Mexican Restaurant,Coffee Shop,Bank,Rental Car Location,Bus Station,Burger Joint,Food,Greek Restaurant,Gym / Fitness Center,Harbor / Marina
27,Magnolia,Medium,8,Mexican Restaurant,Coffee Shop,Bank,Rental Car Location,Bus Station,Burger Joint,Food,Greek Restaurant,Gym / Fitness Center,Harbor / Marina


In [45]:
city1_merged.loc[city1_merged['Cluster Labels'] == 9, city1_merged.columns[[1] + list(range(5, city1_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Boulevard,Medium,9,Coffee Shop,Nightclub,Bookstore,Donut Shop,Mexican Restaurant,Garden,Thrift / Vintage Store,Speakeasy,Auto Dealership,Food Truck
6,Boulevard,Medium,9,Coffee Shop,Nightclub,Bookstore,Donut Shop,Mexican Restaurant,Garden,Thrift / Vintage Store,Speakeasy,Auto Dealership,Food Truck
36,Monument Avenue,Medium,9,Coffee Shop,Nightclub,Bookstore,Donut Shop,Mexican Restaurant,Garden,Thrift / Vintage Store,Speakeasy,Auto Dealership,Food Truck
36,Monument Avenue,Medium,9,Coffee Shop,Nightclub,Bookstore,Donut Shop,Mexican Restaurant,Garden,Thrift / Vintage Store,Speakeasy,Auto Dealership,Food Truck
38,Near West,Medium,9,Coffee Shop,Nightclub,Bookstore,Donut Shop,Mexican Restaurant,Garden,Thrift / Vintage Store,Speakeasy,Auto Dealership,Food Truck
38,Near West,Medium,9,Coffee Shop,Nightclub,Bookstore,Donut Shop,Mexican Restaurant,Garden,Thrift / Vintage Store,Speakeasy,Auto Dealership,Food Truck
38,Near West,Medium,9,Coffee Shop,Nightclub,Bookstore,Donut Shop,Mexican Restaurant,Garden,Thrift / Vintage Store,Speakeasy,Auto Dealership,Food Truck
39,North,Medium,9,Thrift / Vintage Store,Coffee Shop,Nightclub,Bookstore,Donut Shop,Discount Store,Mexican Restaurant,Chinese Restaurant,Casino,Fried Chicken Joint
39,North,Medium,9,Thrift / Vintage Store,Coffee Shop,Nightclub,Bookstore,Donut Shop,Discount Store,Mexican Restaurant,Chinese Restaurant,Casino,Fried Chicken Joint
39,North,Medium,9,Thrift / Vintage Store,Coffee Shop,Nightclub,Bookstore,Donut Shop,Discount Store,Mexican Restaurant,Chinese Restaurant,Casino,Fried Chicken Joint


In [46]:
city2_merged.loc[city2_merged['Cluster Labels'] == 9, city2_merged.columns[[1] + list(range(5, city2_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,El Pinal,Medium,9,Gas Station,Taco Place,Business Service,College Baseball Diamond,Coffee Shop,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store
12,El Pinal,Medium,9,Gas Station,Taco Place,Business Service,College Baseball Diamond,Coffee Shop,Food,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Department Store
44,South Stockton,Medium,9,Gas Station,Café,Pizza Place,Bakery,Grocery Store,Liquor Store,Mexican Restaurant,American Restaurant,Taco Place,Sandwich Place
44,South Stockton,Medium,9,Gas Station,Café,Pizza Place,Bakery,Grocery Store,Liquor Store,Mexican Restaurant,American Restaurant,Taco Place,Sandwich Place
44,South Stockton,Medium,9,Gas Station,Café,Pizza Place,Bakery,Grocery Store,Liquor Store,Mexican Restaurant,American Restaurant,Taco Place,Sandwich Place
44,South Stockton,Medium,9,Gas Station,Café,Pizza Place,Bakery,Grocery Store,Liquor Store,Mexican Restaurant,American Restaurant,Taco Place,Sandwich Place
44,South Stockton,Medium,9,Gas Station,Café,Pizza Place,Bakery,Grocery Store,Liquor Store,Mexican Restaurant,American Restaurant,Taco Place,Sandwich Place
45,Southeast Stockton,Medium,9,Gas Station,Café,Pizza Place,Bakery,Grocery Store,Liquor Store,Mexican Restaurant,American Restaurant,Taco Place,Sandwich Place
45,Southeast Stockton,Medium,9,Gas Station,Café,Pizza Place,Bakery,Grocery Store,Liquor Store,Mexican Restaurant,American Restaurant,Taco Place,Sandwich Place
45,Southeast Stockton,Medium,9,Gas Station,Café,Pizza Place,Bakery,Grocery Store,Liquor Store,Mexican Restaurant,American Restaurant,Taco Place,Sandwich Place


In [47]:
city1_merged.loc[city1_merged['Cluster Labels'] == 10, city1_merged.columns[[1] + list(range(5, city1_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Court End,Medium,10,History Museum,Music Venue,Café,Fast Food Restaurant,College Gym,College Cafeteria,Coffee Shop,Food Truck,Bagel Shop,Sandwich Place
17,Court End,Medium,10,History Museum,Music Venue,Café,Fast Food Restaurant,College Gym,College Cafeteria,Coffee Shop,Food Truck,Bagel Shop,Sandwich Place
19,East,Medium,10,Food,Pool,Hotel,College Gym,College Cafeteria,Coffee Shop,Grocery Store,Music Venue,Café,Gift Shop
19,East,Medium,10,Food,Pool,Hotel,College Gym,College Cafeteria,Coffee Shop,Grocery Store,Music Venue,Café,Gift Shop
19,East,Medium,10,Food,Pool,Hotel,College Gym,College Cafeteria,Coffee Shop,Grocery Store,Music Venue,Café,Gift Shop
46,Tobacco Row,Medium,10,Music Venue,Playground,Garden,German Restaurant,Gift Shop,Grocery Store,Deli / Bodega,College Gym,College Cafeteria,Coffee Shop
46,Tobacco Row,Medium,10,Music Venue,Playground,Garden,German Restaurant,Gift Shop,Grocery Store,Deli / Bodega,College Gym,College Cafeteria,Coffee Shop


In [48]:
city2_merged.loc[city2_merged['Cluster Labels'] == 10, city2_merged.columns[[1] + list(range(5, city2_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,East Stockton,Medium,10,Gas Station,Pizza Place,Fast Food Restaurant,Japanese Restaurant,Creperie,Grocery Store,Turkish Restaurant,Fried Chicken Joint,French Restaurant,Cosmetics Shop
11,East Stockton,Medium,10,Gas Station,Pizza Place,Fast Food Restaurant,Japanese Restaurant,Creperie,Grocery Store,Turkish Restaurant,Fried Chicken Joint,French Restaurant,Cosmetics Shop
11,East Stockton,Medium,10,Gas Station,Pizza Place,Fast Food Restaurant,Japanese Restaurant,Creperie,Grocery Store,Turkish Restaurant,Fried Chicken Joint,French Restaurant,Cosmetics Shop
11,East Stockton,Medium,10,Gas Station,Pizza Place,Fast Food Restaurant,Japanese Restaurant,Creperie,Grocery Store,Turkish Restaurant,Fried Chicken Joint,French Restaurant,Cosmetics Shop
11,East Stockton,Medium,10,Gas Station,Pizza Place,Fast Food Restaurant,Japanese Restaurant,Creperie,Grocery Store,Turkish Restaurant,Fried Chicken Joint,French Restaurant,Cosmetics Shop
11,East Stockton,Medium,10,Gas Station,Pizza Place,Fast Food Restaurant,Japanese Restaurant,Creperie,Grocery Store,Turkish Restaurant,Fried Chicken Joint,French Restaurant,Cosmetics Shop
16,Fox Creek,Medium,10,Japanese Restaurant,Pizza Place,Fast Food Restaurant,Creperie,Convention Center,Hardware Store,Cosmetics Shop,Clothing Store,Big Box Store,Supermarket
16,Fox Creek,Medium,10,Japanese Restaurant,Pizza Place,Fast Food Restaurant,Creperie,Convention Center,Hardware Store,Cosmetics Shop,Clothing Store,Big Box Store,Supermarket
23,La Morada,High,10,Fast Food Restaurant,Fried Chicken Joint,Big Box Store,Hardware Store,Cosmetics Shop,Pizza Place,Coffee Shop,Food,Financial or Legal Service,Filipino Restaurant
34,Northeast Stockton,Medium,10,Pizza Place,Fast Food Restaurant,Gas Station,Japanese Restaurant,Turkish Restaurant,Fried Chicken Joint,French Restaurant,Financial or Legal Service,Department Store,Creperie


In [49]:
city1_merged.loc[city1_merged['Cluster Labels'] == 11, city1_merged.columns[[1] + list(range(5, city1_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Blackwell,Medium,11,Park,Harbor / Marina,Volleyball Court,Caribbean Restaurant,Grocery Store,Seafood Restaurant,Donut Shop,Deli / Bodega,Diner,Discount Store
7,Broad Rock,Medium,11,Park,Volleyball Court,Auto Dealership,Caribbean Restaurant,Seafood Restaurant,Grocery Store,Moving Target,Harbor / Marina,Platform,Wine Shop
7,Broad Rock,Medium,11,Park,Volleyball Court,Auto Dealership,Caribbean Restaurant,Seafood Restaurant,Grocery Store,Moving Target,Harbor / Marina,Platform,Wine Shop
32,Manchester,Medium,11,Park,Harbor / Marina,Volleyball Court,Caribbean Restaurant,Grocery Store,Seafood Restaurant,Donut Shop,Deli / Bodega,Diner,Discount Store


In [50]:
city2_merged.loc[city2_merged['Cluster Labels'] == 11, city2_merged.columns[[1] + list(range(5, city2_merged.shape[1]))]]

,Neighborhood,MedInc,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Gleason Park,Medium,11,Gas Station,Taco Place,Bakery,Smoke Shop,Harbor / Marina,Hotel,Convenience Store,Coffee Shop,Filipino Restaurant,Fast Food Restaurant
19,Gleason Park,Medium,11,Gas Station,Taco Place,Bakery,Smoke Shop,Harbor / Marina,Hotel,Convenience Store,Coffee Shop,Filipino Restaurant,Fast Food Restaurant
19,Gleason Park,Medium,11,Gas Station,Taco Place,Bakery,Smoke Shop,Harbor / Marina,Hotel,Convenience Store,Coffee Shop,Filipino Restaurant,Fast Food Restaurant
26,Louis Park,Medium,11,College Baseball Diamond,Bakery,Smoke Shop,Harbor / Marina,Hotel,Business Service,Convenience Store,Wine Shop,Financial or Legal Service,Filipino Restaurant
26,Louis Park,Medium,11,College Baseball Diamond,Bakery,Smoke Shop,Harbor / Marina,Hotel,Business Service,Convenience Store,Wine Shop,Financial or Legal Service,Filipino Restaurant
43,Seaport District,Medium,11,Bakery,Smoke Shop,Harbor / Marina,Hotel,Café,Convenience Store,Wine Shop,College Baseball Diamond,Financial or Legal Service,Filipino Restaurant
43,Seaport District,Medium,11,Bakery,Smoke Shop,Harbor / Marina,Hotel,Café,Convenience Store,Wine Shop,College Baseball Diamond,Financial or Legal Service,Filipino Restaurant
43,Seaport District,Medium,11,Bakery,Smoke Shop,Harbor / Marina,Hotel,Café,Convenience Store,Wine Shop,College Baseball Diamond,Financial or Legal Service,Filipino Restaurant
